In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
from datetime import datetime
import numpy as np
import timeit
import json

In [2]:
start_time = timeit.default_timer()


df_2019_march = pd.read_json('json_datasets/Kickstarter_2019-03-14T03_20_12_200Z.json', lines=True)


elapsed = timeit.default_timer() - start_time
print(f'time: {elapsed}')

time: 248.26264487599997


In [3]:
# Save the creation datetime for calculating age of project later
df_2019_march_collected_on = datetime(2019, 3, 16)

In [4]:
# clean up the format and make a deep copy
# start_time = timeit.default_timer()


df_2019_march = df_2019_march['data'].apply(pd.Series)
# df_2019_march_copy = df_2019_march.copy(deep=True)
print(f'Total_rows: {len(df_2019_march)}')


# elapsed = timeit.default_timer() - start_time
# print(f'time: {elapsed}')

df_2019_march.head(3)

Total_rows: 209222


,id,photo,name,blurb,goal,pledged,state,slug,disable_communication,country,currency,currency_symbol,currency_trailing_code,deadline,state_changed_at,created_at,launched_at,staff_pick,is_starrable,backers_count,static_usd_rate,usd_pledged,converted_pledged_amount,fx_rate,current_currency,usd_type,creator,location,category,profile,spotlight,urls,source_url,friends,is_starred,is_backing,permissions
0,287514992,{'key': 'assets/011/625/534/5bea1760d7f20943c4...,New Final Round Album,2006 was almost 7 years ago.... Can you believ...,200.0,802.0,successful,new-final-round-album,False,US,USD,$,True,1391899046,1391899046,1387659690,1388011046,False,False,21,1.0,802.0,802,1.0,USD,international,"{'id': 1495925645, 'name': 'Daniel', 'is_regis...","{'id': 2379574, 'name': 'Chicago', 'slug': 'ch...","{'id': 43, 'name': 'Rock', 'slug': 'music/rock...","{'id': 822687, 'project_id': 822687, 'state': ...",True,{'web': {'project': 'https://www.kickstarter.c...,https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN
1,385129759,{'key': 'assets/024/033/030/dea4e3901d10195b03...,Princess Pals Enamel Pin Series,An adorable fantasy enamel pin series of princ...,400.0,2259.0,successful,princess-pals-enamel-pin-series,False,US,USD,$,True,1551801611,1551801611,1549659768,1550073611,False,False,97,1.0,2259.0,2259,1.0,USD,international,"{'id': 1175589980, 'name': 'Katherine', 'slug'...","{'id': 2486340, 'name': 'Sacramento', 'slug': ...","{'id': 54, 'name': 'Mixed Media', 'slug': 'art...","{'id': 3665299, 'project_id': 3665299, 'state'...",True,{'web': {'project': 'https://www.kickstarter.c...,https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN
2,681033598,{'key': 'assets/014/262/672/97944960ba30239051...,Their Life Through Their Lens-the Amish and Me...,Helping a community come together to set the s...,27224.0,29638.0,successful,their-life-through-their-lens-the-amish-and-me...,False,US,USD,$,True,1480607930,1480607932,1477242384,1478012330,True,False,88,1.0,29638.0,29638,1.0,USD,international,"{'id': 1196856269, 'name': 'MelissaThomas', 'i...","{'id': 2383660, 'name': 'Columbus', 'slug': 'c...","{'id': 280, 'name': 'Photobooks', 'slug': 'pho...","{'id': 2730508, 'project_id': 2730508, 'state'...",True,{'web': {'project': 'https://www.kickstarter.c...,https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN


In [5]:
# # Check if data has same features
# df_2019_march_cols = df_2019_march.columns
# columns_original  = ['id', 'photo', 'name', 'blurb', 'goal', 'pledged', 'state', 'slug', 'disable_communication', 'country', 'country_displayable_name', 'currency', 'currency_symbol', 'currency_trailing_code', 'deadline', 'state_changed_at', 'created_at', 'launched_at', 'staff_pick', 'is_starrable', 'backers_count', 'static_usd_rate', 'usd_pledged', 'converted_pledged_amount', 'fx_rate', 'current_currency', 'usd_type', 'creator', 'location', 'category', 'profile', 'spotlight', 'urls', 'source_url', 'friends', 'is_starred', 'is_backing', 'permissions']
# diff = list(set(df_2019_march_cols) - set(columns_original))
# diff2 = list(set(columns_original) - set(df_2019_march_cols))
# print(diff)
# print(diff2)

In [6]:
# # inspect canceled and suspended states
# df_2019_march.state.value_counts()

In [7]:
# drop canceled and suspended states
df_2019_march.drop((df_2019_march[((df_2019_march.state == 'suspended') | 
                                         (df_2019_march.state == 'canceled'))].index), inplace=True)

# check for duplicates
print(f"Rows with duplicate project_id: {df_2019_march.duplicated(['id']).sum()}")

# drop duplicates
df_2019_march.drop_duplicates(['id'], inplace=True)
print(f'Total_rows after dropping duplicates and suspended/canceled states: {len(df_2019_march)}')


Rows with duplicate project_id: 26958
Total_rows after dropping duplicates and suspended/canceled states: 173017


In [8]:
# # get links to project and creator pages
# start_time = timeit.default_timer()


# df_2019_march['project_link'] = 'no_val'
# project_link_loc = df_2019_march.columns.get_loc('project_link')
# for n in range(len(df_2019_march)):
#     project_link = df_2019_march.iloc[n].urls['web']['project']
#     df_2019_march.iat[n, project_link_loc] = project_link
    
# df_2019_march['creator_link'] = 'no_val'
# creator_link_loc = df_2019_march.columns.get_loc('creator_link')
# for n in range(len(df_2019_march)):
#     creator_link = df_2019_march.iloc[n].creator['urls']['web']['user']
#     df_2019_march.iat[n, creator_link_loc] = creator_link  
    
    
# elapsed = timeit.default_timer() - start_time
# print(f'time: {elapsed}')

In [9]:
# # get main_category id
# start_time = timeit.default_timer()


# categories_map = {1:'Art',3:'Comics', 17:'Theater', 15:'Photography', 
#                 7:'Design', 6: 'Dance', 11:'Film & Video', 12:'Games', 16:'Technology', 
#                 13:'Journalism', 18:'Publishing', 9:'Fashion', 10:'Food', 14:'Music', 26:'Crafts'}
# categories_df = pd.DataFrame(categories_map, index=[0])


# df_2019_march['main_category'] = 'no_value'
# cat_loc = df_2019_march.columns.get_loc('main_category')


# for n in (range(len(df_2019_march))):
#     if 'parent_id' not in df_2019_march.iloc[n]['category'].keys():
#         cat_id = df_2019_march.iloc[n]['category']['id']
#         df_2019_march.iat[n, cat_loc] = cat_id
#     elif pd.isnull(df_2019_march.iloc[n]['category']['parent_id']):
#         cat_id = df_2019_march.iloc[n]['category']['id']
#         df_2019_march.iat[n, cat_loc] = cat_id
#     else:
#         par_id = df_2019_march.iloc[n]['category']['parent_id']
#         df_2019_march.iat[n, cat_loc] = par_id
        
# df_2019_march['main_category'] = df_2019_march['main_category'].apply(lambda x: categories_df[x])


# elapsed = timeit.default_timer() - start_time
# print(f'time: {elapsed}')

# df_2019_march.head(1)

In [10]:
# # keep these features
# df_main_feats = ['id', 'name', 'blurb', 'goal', 'pledged', 'state', 'country',
# 'currency', 'deadline', 'launched_at', 'staff_pick', 'main_category',
# 'backers_count', 'static_usd_rate', 'usd_pledged' ]
# # Drop these 
# df_drop_feats = list(set(df_2019_march_cols) - set(df_main_feats))
# print(f'Columns removed: {df_drop_feats}')

# # drop features
# df_2019_march.drop(df_drop_feats, axis=1, inplace=True)

In [11]:
# convert unix dates to datetime objects
df_2019_march.deadline = df_2019_march.deadline.apply(lambda x: (datetime.fromtimestamp(x)))
df_2019_march.launched_at = df_2019_march.launched_at.apply(lambda x: (datetime.fromtimestamp(x)))

In [12]:
# rename columns
df_2019_march.rename(columns={'id':'project_id'}, inplace=True)
df_2019_march.rename(columns={'name':'project_name'}, inplace=True)
df_2019_march.rename(columns={'blurb':'project_description'}, inplace=True)
df_2019_march.rename(columns={'state':'outcome_state'}, inplace=True)


In [13]:
# add project duration feature
df_2019_march['duration'] = df_2019_march.deadline - df_2019_march.launched_at
df_2019_march['days_from_launch'] =(df_2019_march_collected_on - df_2019_march.launched_at).apply(lambda x: x.days)

# # add feature 'in_mid_duration_range' indicating whether project is about halfway through
# df_2019_march['duration_mid_point'] = df_2019_march.duration.apply(lambda x: x.days) / 2
# df_2019_march['lower_bound_mid_point_range'] = df_2019_march.duration_mid_point.apply(lambda x: x - 5)
# df_2019_march['upper_bound_mid_point_range'] = df_2019_march.duration_mid_point.apply(lambda x: x + 5)
# df_2019_march['in_mid_duration_range'] = df_2019_march.days_from_launch.between(df_2019_march.lower_bound_mid_point_range, df_2019_march.upper_bound_mid_point_range)

# df_2019_march.drop(['duration_mid_point', 'lower_bound_mid_point_range', 'upper_bound_mid_point_range'], axis=1, inplace=True)


In [14]:
# df_2019_march['duration'][0].quantile(.1)
df_2019_march['duration_10_percent'] = df_2019_march.duration.apply(lambda x: x.days / 10 ) 


In [15]:
# add feature 'in_mid_duration_range' indicating whether project is about halfway through
df_2019_march['lower_bound'] = df_2019_march.duration_10_percent.apply(lambda x: x if x < 5 else x - 5)
df_2019_march['upper_bound'] = df_2019_march.duration_10_percent.apply(lambda x: x + 5)
df_2019_march['in_range'] = df_2019_march.days_from_launch.between(df_2019_march.lower_bound, df_2019_march.upper_bound)

# df_2019_march.drop(['duration_mid_point', 'lower_bound_mid_point_range', 'upper_bound_mid_point_range'], axis=1, inplace=True)

In [16]:
# calculate usd_goal
df_2019_march['usd_goal'] = df_2019_march.goal * df_2019_march.static_usd_rate


In [17]:
# Replace goal and rearrange
# col_list = df_2019_march.columns.to_list()
# print(col_list)
col_list_rearrange = ['project_id', 'project_name', 'project_description', 'outcome_state', 'in_range',
                      'duration_10_percent', 'days_from_launch', 
                      'usd_goal', 'usd_pledged', 'currency', 'launched_at', 'deadline', 'staff_pick', 'country', 
                      'duration' , 'backers_count']
df_2019_march = df_2019_march[col_list_rearrange]
df_2019_march.head(1)
                      

,project_id,project_name,project_description,outcome_state,in_range,duration_10_percent,days_from_launch,usd_goal,usd_pledged,currency,launched_at,deadline,staff_pick,country,duration,backers_count
0,287514992,New Final Round Album,2006 was almost 7 years ago.... Can you believ...,successful,False,4.5,1906,200.0,802.0,USD,2013-12-25 17:37:26,2014-02-08 17:37:26,False,US,45 days,21


In [18]:
df_2019_march_copy = df_2019_march.copy(deep=True)
# convert datetime columns before exporting
# df_2019_march.duration = df_2019_march.duration.apply(lambda x: x.days) 
df_2019_march.launched_at = df_2019_march.launched_at.apply(lambda x: x.strftime('%m-%d-%Y'))
df_2019_march.deadline = df_2019_march.deadline.apply(lambda x: x.strftime('%m-%d-%Y'))

In [19]:
# export cleaned dataframe to csv
df_2019_march.to_csv('clean_datasets_10/main_df_2019_march.csv', index=False)